# Mini-project II
The details for the miniproject from Week 2


In this miniproject, we will combine and practice topics that we have learned in previous two weeks:
- APIs
- Databases (SQL)
- Pandas
- Processing special data types in Python

We will work with these APIs:
1. [Foursquare](https://developer.foursquare.com/places) - we have already come across this one
2. [Yelp](https://www.yelp.com/developers/documentation/v3/get_started) - this API offers similar services as Foursquare.
3. (Stretch) [Google Places API](https://developers.google.com/places/web-service/intro) - this google api offers similar service as well.

The main goal of the mini-project is to build the database of restaurants, bars and various points of interest (POIs) in the area of your choice and find out which API has better coverage in the selected area. The APIs have limited number of requests for free, so start with the smaller area. The project consists of following tasks:

- pull the data about various POIs in the area through API. (Search Yelp for companiees that are in the area using [these instructions](https://www.yelp.com/developers/documentation/v3/business_search)). If you run out of requests for any of the APIs, don't worry, it's ok to use only sample data you already have or the POIs from the Yelp API. It's approach and process that counts, not the actual number of places we were able to get.
- create own SQLite database and store the data about the POIs. Think about what will be the best structure of the database. We've used and created sqlite3 databases before in the activity [**SQL in Python**](https://data.compass.lighthouselabs.ca/b9e08cd5-68c6-490c-a32b-a66f01bf53e1).
- compare the results using SQL or Pandas (it's up to you:)) and see which API has a better coverage of the area.
- choose the top 10 POIs based on the popularity (number of reviews or average rating) ([Yelp](https://www.yelp.com/developers/documentation/v3/business), [Foursquare](https://developer.foursquare.com/docs/api-reference/venues/details/)).
- (Stretch) By implementing [travelling salesman problem (TSP)](https://en.wikipedia.org/wiki/Travelling_salesman_problem), how much time would it take to visit all of these places? ([Directions API](https://developers.google.com/maps/documentation/directions/start) from google will be helpful here). We will have to find travel time between all places (top 10). We can use [ortools](https://developers.google.com/optimization/routing/tsp) from Google to effectively implement TSP. These tools are very powerful and [easy to install](https://developers.google.com/optimization/install).

We have a lot of work so let's start right away. Enjoy!!


> #### Note
> Some APIs from Google aren't for free anymore but each account has 200$ credit every month. Therefore we are able to use these services for learning purposes for free.


# Best Places to Visit
### Using Yelp and Foursquare

## Which API is better?
![Yelp](img/yelp-icon.jpg) ![Foursquare](img/foursquare-icon.jpg)

## Today's Itinerary
1. Getting Data from Yelp and Foursquare
2. The Top 10 Places of Interest
3. Which is Better?

In [2]:
import pandas as pd
import numpy as np
import json
import requests as re
from api_keys import foursquare, yelp
from IPython.display import JSON

In [3]:
# setup API ids and passwords
fs_id = foursquare['id']
fs_secret = foursquare['secret']
fs_key = foursquare['key']

In [4]:
il = '49.239971669380836,-122.97203063964844'
ver = '20211124'
query = 'restaurant'
fields = 'fsq_id%2Ccategories%2Cdistance%2Cgeocodes%2Clocation%2Cname%2Ctel%2Crating%2Cstats%2Cpopularity%2Cprice'
# url = "https://api.foursquare.com/v3/places/4abbc832f964a520e08420e3?fields=fsq_id%2Ccategories%2Cdistance%2Cgeocodes%2Clocation%2Cname%2Ctel%2Crating%2Cstats%2Cpopularity%2Cprice"

url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={il}&fields={fields}&limit=50"
headers = {
    "Accept": "application/json",
    "Authorization": fs_key
}
response = re.request("GET", url, headers=headers)

In [5]:
JSON(response.json())

<IPython.core.display.JSON object>

In [6]:
results = response.json()['results']
JSON(results)

<IPython.core.display.JSON object>

In [7]:
fs_locations = pd.DataFrame()
fs_categories = pd.DataFrame()
# locations = pd.DataFrame(columns=['id', 'distance', 'long', 'lat', 'address', 'country', 'cross_street', 'locality', 'region', 'name'])

In [8]:
# for each result, grab all the relevant data
# in the case of the categories, a new dataframe is needed.
# grab the data from each category, along with the fsr_id as a key
# and push to another dataframe
for i in results:
    lon = i['geocodes']['main']['longitude']
    lat = i['geocodes']['main']['latitude']
    addr = i['location']['address']
    cont = i['location']['country']
    cros = ''
    try:
        cros = i['location']['cross_street']
    except:
        print('cross_street missing')
    locality = i['location']['locality']
    region = i['location']['region']
    popularity = i['popularity']
#     try:
#         popularity = i['popularity']
#     except:
#         print('popularity missing')
    price = ''
    try:
        price = i['price']
    except:
        print('price missing')
    rating = ''
    try:
        rating = i['rating']
    except:
        print('rating missing')
    rating_count = ''
    try:
        rating_count = i['stats']['total_ratings']
    except:
        print('stats missing')
    tel = ''
    try:
        tel = i['tel']
    except:
        print('telephone missing')
    fs_locations = fs_locations.append({'fsq_id': i['fsq_id'], 'distance': i['distance'], 'long': lon, 'lat': lat, 
                            'address': addr, 'country': cont, 'cross_street': cros, 'locality': locality,
                            'region': region, 'name': i['name'], 'popularity': popularity,
                            'price': price, 'rating': rating, 'rating_count': rating_count,
                            'telephone': tel}, ignore_index=True)
    for category in i['categories']:
        fs_categories = fs_categories.append({'cat_id': category['id'], 'cat_name': category['name'], 'fsq_id': i['fsq_id']},
                     ignore_index=True)

cross_street missing
telephone missing
price missing
price missing
price missing
cross_street missing


In [9]:
fs_locations = fs_locations.drop_duplicates()

In [10]:
# store the data into the sql database
# import sqlite3
# import sqlalchemy
# engine = sqlalchemy.create_engine('sqlite:///poi_database.db')

In [11]:
# locations.to_sql('foursquare_locations', engine)
# categories.to_sql('foursquare_categories', engine)

### Yelp API

In [12]:
yelp_id = yelp['id']
yelp_key = yelp['key']

In [13]:
# Loading the data
headers = {'Authorization': f'Bearer {yelp_key}'}
url = 'https://api.yelp.com/v3/businesses/search'
lat = 49.28193507156773
lon = -123.00004885392272
params = {'term':'restaurant', 'location': 'Burnaby', 'limit': 50}
yelp_req = re.get(url, params=params, headers=headers)

In [14]:
JSON(yelp_req.json())

<IPython.core.display.JSON object>

In [15]:
yelp_req = yelp_req.json()['businesses']

In [16]:
JSON(yelp_req)

<IPython.core.display.JSON object>

In [17]:
# import pprint
# pp = pprint.PrettyPrinter()
# pp.pprint(yelp_req[0])

In [18]:
yelp_loc = pd.DataFrame()
yelp_cat = pd.DataFrame()

In [19]:
# Same as before with the foursquare api
for i in yelp_req:
    yelp_id = i['id']
    name = i['name']
    lat = i['coordinates']['latitude']
    lon = i['coordinates']['longitude']
    distance = i['distance']
    address = i['location']['address1']
    phone = i['phone']
    city = i['location']['city']
    country = i['location']['country']
    price = ''
    try:
        price = i['price']
    except:
        print('price missing')
    rating = i['rating']
    review_count = i['review_count']
    yelp_loc = yelp_loc.append({'yelp_id': yelp_id, 'name': name, 'lat': lat, 'lon': lon,
                                'distance': distance, 'address': address, 'phone': phone,
                                'city': city, 'country': country, 'price': price,
                                'rating': rating, 'review_count': review_count
                                }, ignore_index=True)
    for cat in i['categories']:
        title = cat['alias']
        yelp_cat = yelp_cat.append({'yelp_id': yelp_id, 'category': title}, 
                                    ignore_index=True)

price missing
price missing
price missing
price missing
price missing
price missing
price missing
price missing
price missing
price missing
price missing
price missing
price missing
price missing
price missing
price missing
price missing
price missing


In [20]:
# save the data into sql
# yelp_loc.to_sql('yelp_locations', engine)
# yelp_cat.to_sql('yelp_categories', engine)

## Gathering Data
1. fetch restaurant info from each API
2. unpack JSON data
3. store data into SQL database

## Comparing the two API's
What data does each provide?

## Data Types Comparison
|Foursquare |Yelp |
|---   |---   |
|   Name  |Name   |
| Address | Address |
| Distance| Distance |
| Coordinates | Coordinates |
| country, province, city | country,province, city|
| Categories | Categories |
| Price   | Price  |
| Ratings   | Ratings |
| Phone Number  | Phone Number |
| Review Count  | Review Count |

## Top 10 Restaurants

In [21]:
yelp_top10 = yelp_loc.sort_values('rating', ascending=False).head(10)

In [22]:
fs_top10 = fs_locations.sort_values('rating', ascending=False).head(10)

In [23]:
print('Yelp')
yelp_top10[['name', 'rating', 'review_count']]

Yelp


,name,rating,review_count
22,Wonderffle,5.0,8.0
16,Henry's Kitchen Pasta & Grill,5.0,46.0
12,Pearl House Burnaby,4.5,11.0
36,Astro's Kitchen,4.5,6.0
1,Little Minh's,4.5,101.0
38,Joojak Restaurant,4.5,123.0
26,Fat Boy Kitchen,4.5,10.0
40,IT Char Burger,4.5,62.0
33,Siam Le Bien,4.5,47.0
14,Wild Thyme,4.5,311.0


In [24]:
print('Foursquare')
fs_top10[['name', 'rating', 'rating_count']]

Foursquare


,name,rating,rating_count
9,Camellia Tea & Coffee,8.6,60.0
36,Mon Paris Patisserie Inc,8.6,19.0
17,Hi Genki Japanese Restaurant,8.4,90.0
24,Jinya Ramen Bar,8.3,28.0
8,Myst Asian Fusion,8.3,20.0
16,El Comal,8.2,13.0
23,Me-N-Ed's Pizza Parlor,8.2,40.0
22,Kea's Meats,8.0,12.0
43,Caritas 9 Coffee Roasters,8.0,9.0
30,Cactus Club Cafe Station Square,8.0,4.0


## Ease of Use
Both API's were of similar ease of use.
- Decently written documentation
- Provided examples
- Testing within documentation
- API functions were relatively easy to use

## Options and Functionality
- Yelp: Less functions, more comprehensive data
- Yelp: Less options on what data to return
- Foursquare: More options for more granular data
- Foursquare: the options were a bit difficult to find

## Data Quality
- Foursquare: Some missing data
    - Some missing data, different types
    - Missing data is not passed through the API, can cause data read problems
- Yelp: Few Errors with missing data
    - Most missing data filled in with blanks
    - Only missing price data caused issue

## Which is better?
I would choose the Foursquare API
- Relatively simple API
- Comprehensive options for fetching data
- Somewhat less missing data

# The End!
Questions?